In [2]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf


In [3]:

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="best-fp16.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Define function to find nearest landmark to a point
def find_nearest_landmark(point, landmarks):
    distances = np.linalg.norm(landmarks - point, axis=1)
    nearest_idx = np.argmin(distances)
    return landmarks[nearest_idx]

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    # Read frame from webcam
    ret, frame = cap.read()

    # Run MediaPipe Pose model to get landmarks
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    landmarks = np.array([[landmark.x, landmark.y] for landmark in results.pose_landmarks.landmark])

    # Run TFLite model to predict wound location and name
    input_data = np.expand_dims(frame, axis=0)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    wound_location = output_data[0][:2]
    wound_name = output_data[0][2]

    # Draw red bounding box around wound location
    x, y = map(int, wound_location)
    cv2.rectangle(frame, (x-20, y-20), (x+20, y+20), (0, 0, 255), 2)

    # Find 2 landmarks near wound location
    nearest_landmarks = np.array([
        find_nearest_landmark(wound_location, landmarks),
        find_nearest_landmark(wound_location + np.array([0, -50]), landmarks)
    ])

    # If wound is a bite wound, find nearest landmark near shoulder or hip
    if wound_name == 'bite_wound':
        candidate_landmarks = np.array([
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks),
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value], landmarks),
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value], landmarks),
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value], landmarks)
        ])
        nearest_landmarks[1] = find_nearest_landmark(nearest_landmarks[0], candidate_landmarks)

    # Calculate midpoint between wound location and nearest landmark
    midpoint = np.mean(nearest_landmarks, axis=0)

    # Draw red circle at midpoint
    cv2.circle(frame, tuple(map(int, midpoint)), 10, (0, 0, 255), -1)

    # Show frame
    cv2.imshow('Frame', frame)

    # Exit if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


ValueError: Cannot set tensor: Got value of type UINT8 but expected type FLOAT32 for input 0, name: serving_default_input_1:0 